Mount drive

In [1]:
import os
from google.colab import drive
drive.mount('/content/gdrive')

path = '/content/gdrive/MyDrive/DeXpression'
os.chdir(path)

DATA_FOLDER = '/content/gdrive/MyDrive/DeXpression/dataset/CK+48'

Mounted at /content/gdrive


Standard imports


In [2]:
import numpy as np

import torch

from torchsummary import summary

from torch.nn import Module, Conv2d, MaxPool2d, Linear, ReLU, LogSoftmax

from torch.nn import LayerNorm, BatchNorm2d, Dropout

import torch.nn.functional as F
import torch.utils.data as data
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms

#DeXpression model



Transfer to GPU (if applicable)

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
class DeXpression(Module):
  
  def __init__(self):
    super(DeXpression, self).__init__()

    # block-1
    self.conv1 = Conv2d(in_channels=1, out_channels=64, kernel_size=7, stride=2, padding=3)
    self.pool1 = MaxPool2d(kernel_size=3, stride=2, padding=0)
    self.lrn1  = LayerNorm([64, 55, 55])

    # feature extractor 1
    self.conv2a = Conv2d(in_channels=64, out_channels=96, kernel_size=1, stride=1, padding=0)
    self.conv2b = Conv2d(in_channels=96, out_channels=208, kernel_size=3, stride=1, padding=1)
    self.pool2a = MaxPool2d(kernel_size=3, stride=1, padding=1)
    self.conv2c = Conv2d(in_channels=64, out_channels=64, kernel_size=1, stride=1, padding=0)
    self.pool2b = MaxPool2d(kernel_size=3, stride=2, padding=0)

    # feature extractor 2
    self.conv3a = Conv2d(in_channels=272, out_channels=96, kernel_size=1, stride=1, padding=0)
    self.conv3b = Conv2d(in_channels=96, out_channels=208, kernel_size=3, stride=1, padding=1)
    self.pool3a = MaxPool2d(kernel_size=3, stride=1, padding=1)
    self.conv3c = Conv2d(in_channels=272, out_channels=64, kernel_size=1, stride=1, padding=0)
    self.pool3b = MaxPool2d(kernel_size=3, stride=2, padding=0)

    # fully connected layer 
    self.fc                  = Linear(in_features=272*13*13, out_features=7)
    self.softmax             = LogSoftmax(dim=1)
    self.batch_normalization = BatchNorm2d(272)
    self.dropout             = Dropout(p=0.2)

  def forward(self, x, dropout=True, batch_normalization=True):
    """
    Perform a single step of forward propogation
    """
    # block-1
    conv1_out = F.relu(self.conv1(x))
    pool1_out = self.pool1(conv1_out)
    lrn1_out  = self.lrn1(pool1_out)

    # feature extractor 1
    # branch 1
    conv2a_out = F.relu(self.conv2a(lrn1_out))
    conv2b_out = F.relu(self.conv2b(conv2a_out))
    # branch 2
    pool2a_out = self.pool2a(lrn1_out)
    conv2c_out = F.relu(self.conv2c(pool2a_out))
    # concatenate both branches
    concat2_out = torch.cat((conv2b_out, conv2c_out), 1)
    pool2b_out  = self.pool2b(concat2_out)

    # feature extractor 2
    # branch 1
    conv3a_out = F.relu(self.conv3a(pool2b_out))
    conv3b_out = F.relu(self.conv3b(conv3a_out))
    # branch 2
    pool3a_out = self.pool3a(pool2b_out)
    conv3c_out = F.relu(self.conv3c(pool3a_out))
    # concatenate both branches
    concat3_out = torch.cat((conv3b_out, conv3c_out), 1)
    pool3b_out  = self.pool3b(concat3_out)

    # dropout enabled
    if dropout:
      pool3b_out = self.dropout(pool3b_out)
    
    # batch normalization enabled
    if batch_normalization:
      pool3b_out = self.batch_normalization(pool3b_out)

    pool3b_shape = pool3b_out.shape
    pool3b_flat  = pool3b_out.reshape([-1, pool3b_shape[1] * pool3b_shape[2] * pool3b_shape[3]])

    output = self.fc(pool3b_flat)
    logits = self.softmax(output)

    return logits


Utility functions to read data

In [5]:
import glob
from pathlib import Path
from matplotlib import image as im


def data(*paths):
  return DATA_FOLDER.joinpath(*paths)


def read_file(img_file, label_file):
  """
  Read and convert images to numpy arrays
  """
  image = im.imread(img_file)

  with open(label_file, "r") as file:
        label = float(file.read())

  return image, label


def load_from_array():
  """
  Load dataset from a specified folder
  """
  x = np.load(data("x.npy")).reshape(-1, 1, 224, 224)
  y = np.load(data("y.npy"))

  return x, y


def save_to_array():
  """
  Save dataset to a specified folder
  """
  with open(data("x.npy"), "wb") as file:
    np.save(file, x)

  with open(data("y.npy"), "wb") as file:
    np.save(file, y)


def load_dataset(use_existing = True):
  """
  Return input and output variables from the
  dataset
  """
  if use_existing:
    x, y = load_from_array()
  
  else:
    data_dir = DATA_FOLDER
    images   = []
    labels   = []

    for image_file in sorted(glob.glob(f"{data_dir}/**/*.png")):
      image_path = os.path.dirname(image_file)
      label_path = image_path.replace("images", "labels")

      if os.path.exists(label_path):
        if not len(os.listdir(label_path)) == 0:
          label_file = os.path.join(label_path, os.listdir(label_path)[0])

          image, label = file_reader(image_file, label_file)
          images.append(image)
          labels.append(label)

    x = np.stack(images, axis = 0).reshape(-1, 1, 224, 224)
    y = np.stack(labels, axis = 0)

    save_to_array(x, y)


  print("Loaded datasets {} and {}".format(x.shape, y.shape))
  print("")

  return x, y

Dataset utilities (Shuffle and convert to torch tensor)

In [6]:
from sklearn.model_selection import KFold
from sklearn.utils import shuffle as s


def kfold(x, y, splits = 5, shuffle = True):
  """
  Perform a K-fold split on the dataset
  x -> Input variables from the dataset
  y -> Output variables frm the dataset
  """
  x, y = s(x, y)
  kfold = KFold(n_splits = splits, shuffle = shuffle)

  for train, test in kfold.split(x, y):
    x_train, y_train = x[train], y[train]
    x_test, y_test   = x[test], y[test]

    yield x_train, y_train, x_test, y_test


def convert_to_torch(x_train, y_train, x_test, y_test):
  """
  Convert the train and test data to torch tensors
  """
  # convert training images to a torch tensor
  x_train = torch.from_numpy(x_train)
  x_train = x_train.type(torch.FloatTensor)

  # convert training labels to a torch tensor
  y_train = y_train.astype(int)
  y_train = torch.from_numpy(y_train)

  # convert test images to torch tensor
  x_test = torch.from_numpy(x_test)
  x_test = x_test.type(torch.FloatTensor)

  # convert testing labels to torch tensor
  y_test = y_test.astype(int)
  y_test = torch.from_numpy(y_test)

  return x_train, y_train, x_test, y_test
